In [20]:
from irec.recommendation.agents.value_functions import LinearUCB, MostPopular, GenericThompsonSampling
from irec.recommendation.agents.action_selection_policies import ASPGenericGreedy, ASPGreedy
from irec.offline_experiments.metric_evaluators import UserCumulativeInteraction
from irec.offline_experiments.evaluation_policies import FixedInteraction
from irec.recommendation.agents import SimpleEnsembleAgent, SimpleAgent
from irec.offline_experiments.metrics import Hits
from irec.environment.loader import FullData

In [2]:
import pandas as pd
import numpy as np

## Load Dataset

In [3]:
# Dataset
dataset = {
    'path': "../datasets/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "global", 'train_size': 0.8, 'test_consumes': 5}
validation = {'validation_size': 0.2}
# Loader
loader = FullData(dataset, splitting, validation)
train_dataset, test_dataset, x_validation, y_validation = loader.process()


Applying splitting strategy: global

Test shape: (16892, 4)
Train shape: (80393, 4)

Generating x_validation and y_validation: 
Test shape: (15729, 4)
Train shape: (61345, 4)


## Creating the agents

In [12]:
params = {
    "LinearUCB": {"alpha": 1.0, "item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "MostPopular": {},
    "GenericThompsonSampling": {"alpha_0": {"LinearUCB": 100, "MostPopular": 1}, "beta_0": {"LinearUCB": 100, "MostPopular": 1}},
}

### Creating the simple agents

In [ ]:
vf1 = LinearUCB(**params["LinearUCB"])
vf2 = MostPopular(**params["MostPopular"])

asp_sa = ASPGreedy()

agent1 = SimpleAgent(vf1, asp_sa, name="LinearUCB")
agent2 = SimpleAgent(vf2, asp_sa, name="MostPopular")

### Creating the Ensemble Agent

In [13]:
asp_sea = ASPGenericGreedy()
vf_sea = GenericThompsonSampling(**params["GenericThompsonSampling"])
ensemble_agent = SimpleEnsembleAgent(agents=[agent1, agent2], action_selection_policy=asp_sea, name="EnsebleAgent", value_function=vf_sea)

In [14]:
agents = [agent1, agent2, ensemble_agent]

## Getting the recommendations

In [15]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [16]:
interactions = {}
for agent in agents:
    print(agent.name)
    agent_interactions, action_info = eval_policy.evaluate(agent, train_dataset, test_dataset)
    interactions[agent.name] = agent_interactions

LinearUCB
Starting LinearUCB Training


rmse=0.800: 100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


Ended LinearUCB Training


LinearUCB: 100%|██████████| 18900/18900 [00:17<00:00, 1083.03it/s]


MostPopular
Starting MostPopular Training
Ended MostPopular Training


MostPopular: 100%|██████████| 18900/18900 [00:02<00:00, 6731.79it/s]


EnsebleAgent
Starting EnsebleAgent Training


rmse=0.801: 100%|██████████| 20/20 [00:18<00:00,  1.08it/s]


Ended EnsebleAgent Training


EnsebleAgent: 100%|██████████| 18900/18900 [00:22<00:00, 844.46it/s] 


### Evaluating the models

In [17]:
# Cumulative Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [18]:
# Getting the results
cumulative_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    cumulative_results[agent_name] = hits_values


Evaluating LinearUCB

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.86 seconds executing Hits metric

Evaluating MostPopular

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.38 seconds executing Hits metric

Evaluating EnsebleAgent

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing

In [19]:
df_cumulative = pd.DataFrame(columns=["Model", 5, 10, 20, 50, 100])
df_cumulative["Model"] = list(cumulative_results.keys())
df_cumulative.set_index("Model", inplace=True)
for agent_name, results in cumulative_results.items():
    df_cumulative.loc[agent_name] = [
        np.mean(list(metric_values.values())) for metric_values in results
    ]
df_cumulative

,5,10,20,50,100
Model,,,,,
LinearUCB,1.936508,3.47619,6.333333,14.084656,22.915344
MostPopular,1.666667,2.904762,5.126984,10.063492,16.703704
EnsebleAgent,1.846561,3.470899,6.492063,14.132275,23.026455
